In [1]:
import requests
import json
import yaml
import ipaddress
import sys
import os
from pathlib import Path
import subprocess
import copy
#as_set = os.environ["AS_SET_SRC"]
#as_dst = os.environ["AS_SET_SRC"]
#password = os.environ["RIPE_PASSWD"]

In [2]:
as_set_src = ""
as_set_dst = ""
password = ""
max_depth = 3
dst_cache_file = Path("cache/dst_cache.json")
irrdb = "RIPE,APNIC,AFRINIC,ARIN,LACNIC,RADB"

In [3]:
src_url = f"https://rest.db.ripe.net/ripe/as-set/{as_set_src}"
dst_url = f"https://rest.db.ripe.net/ripe/as-set/{as_set_dst}?password={password}"
t1_asns = [ 701, 1239, 1299, 2914, 3257, 3320, 3356, 3491, 5511, 6453, 6461, 6762, 6830, 7018, 12956, 174, 1273, 2828, 4134, 4809, 4637, 6939, 7473, 7922, 9002 ]

headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}


In [4]:
def extract_member(base_json):
    return list(map(lambda x:x["value"],filter(lambda x:x["name"] == "members", base_json["objects"]["object"][0]["attributes"]["attribute"])))

def index_of_first(lst, pred):
    for i, v in enumerate(lst):
        if pred(v):
            return i
    return 1

def pack_member(base_json,member_list):
    base_json = copy.deepcopy(base_json)
    old_list = base_json["objects"]["object"][0]["attributes"]["attribute"]
    first_member_idx = index_of_first(old_list,lambda x:x["name"] == "members")
    old_list = list(filter(lambda x:x["name"] != "members",old_list))
    new_list = old_list[0:first_member_idx] + [{"name": "members", "value": member, "referenced-type":"aut-num" if member[:2] == "AS" and member[2:].isdecimal() else "as-set" } for member in member_list] + old_list[first_member_idx:]
    base_json["objects"]["object"][0]["attributes"]["attribute"] = new_list
    return base_json
def getval(strin):
    return strin.split(":",1)[1].strip()

In [5]:
src_json = json.loads(requests.request("GET", src_url, headers=headers).text)
src_asset = extract_member(src_json)
if dst_cache_file.is_file():
    dst_json = json.loads(open(dst_cache_file).read())
else:
    dst_json = json.loads(requests.request("GET", dst_url, headers=headers).text)
dst_asset = extract_member(dst_json)


In [6]:
flatted_members = {}
for as_set in src_asset:
    #print(as_set)
    query = ["bgpq4", "-tj","-L",str(max_depth), "-S",irrdb , as_set]
    #print(" ".join(query))
    bgpq4_asns = subprocess.run(query, stdout=subprocess.PIPE).stdout.decode()
    asset_asns = json.loads(bgpq4_asns)["NN"]
    asset_t1s = sorted(set(asset_asns) & set(t1_asns))
    if len(asset_t1s)>0:
        print(f"Warning: {as_set} contains t1_asns:{asset_t1s}")
        continue
    for asn in asset_asns:
        flatted_members[asn] = 0

In [7]:
flatted_members = sorted(list(flatted_members.keys()))

In [8]:
if dst_asset != flatted_members:
    dst_json_new = pack_member(dst_json,map(lambda x:"AS" + str(x),flatted_members))
    payload = json.dumps(dst_json_new)
    response = requests.request("PUT", dst_url, headers=headers, data=payload)
    response.raise_for_status()
    dst_json_new = json.loads(response.text)
    print("updated:",as_set_dst)
else:
    print("same, no update:",as_set_dst)

updated: AS203472:AS-DST


In [ ]:
dst_json_new

In [ ]:
dst_json